In [57]:
# Paso 1: Cargar los datos
data = pd.read_csv('angelsdata.csv')


/tmp/ipykernel_2937/3942705368.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('angelsdata.csv')


In [62]:
# Cargar los datos
data = pd.read_csv('angelsdata.csv', low_memory=False)

# Imprimir los nombres de las columnas
print(data.columns)


Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import LabelEncoder

# Cargar los datos
data = pd.read_csv('angelsdata.csv', low_memory=False)

# Eliminar columnas innecesarias
data = data.drop(columns=['DR_NO', 'AREA NAME', 'Rpt Dist No', 'Mocodes', 'Cross Street', 'LAT', 'LON'])

# Convertir la columna de fecha a componentes numéricos
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], errors='coerce')
data['YEAR'] = data['DATE OCC'].dt.year
data['MONTH'] = data['DATE OCC'].dt.month
data['DAY'] = data['DATE OCC'].dt.day
data['HOUR'] = data['DATE OCC'].dt.hour
data = data.drop(columns=['DATE OCC'])

# Seleccionar columnas con pocas categorías para aplicar get_dummies
categorical_columns = ['Status', 'Vict Sex', 'Vict Descent', 'Premis Desc']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Codificar otras columnas categóricas con LabelEncoder
label_encodable_columns = ['Weapon Desc']  # Ajusta según tus necesidades
for col in label_encodable_columns:
    data[col] = LabelEncoder().fit_transform(data[col])

# Verificar y convertir todas las columnas a numéricas
for col in data.columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Rellenar valores NaN que puedan haber resultado de las conversiones
data = data.fillna(0)

# Dividir los datos en características (X) y etiquetas (y)
X = data.drop(columns=['Crm Cd Desc'])  # Asegúrate de que 'Crm Cd Desc' es la etiqueta que quieres predecir
y = data['Crm Cd Desc']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Guardar el modelo entrenado
joblib.dump(model, 'crime_prediction_model.pkl')



/tmp/ipykernel_5323/223245640.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], errors='coerce')


Accuracy: 1.0


['crime_prediction_model.pkl']

In [4]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import pandas as pd

# Cargar el modelo entrenado
model = joblib.load('crime_prediction_model.pkl')

# Definir las columnas esperadas, tal como las configuramos en el taller
model_columns = ['DR_NO', 'AREA NAME', 'Rpt Dist No', 'Mocodes', 'Cross Street', 'LAT', 'LON', 
                 'YEAR', 'MONTH', 'DAY', 'HOUR', 
                 'Vict Sex_M', 'Vict Descent_H', 'Premis Desc_STREET', 
                 'Weapon Desc_HANDGUN', 'Status_IC']

class CrimePredictionInput(BaseModel):
    DATE_OCC: str
    AREA: int
    Crm_Cd: int
    Vict_Age: int
    Vict_Sex: str
    Vict_Descent: str
    Premis_Desc: str
    Weapon_Desc: str
    Status: str

class CrimePredictionOutput(BaseModel):
    PRIMARY_DESCRIPTION: str

app = FastAPI()

@app.post('/predict/', response_model=CrimePredictionOutput)
async def predict_crime(data: CrimePredictionInput):
    try:
        # Convertir los datos de entrada en un DataFrame
        input_data = pd.DataFrame([data.dict()])

        # Preprocesar los datos
        input_data['DATE_OCC'] = pd.to_datetime(input_data['DATE_OCC'], errors='coerce')
        input_data['YEAR'] = input_data['DATE_OCC'].dt.year
        input_data['MONTH'] = input_data['DATE_OCC'].dt.month
        input_data['DAY'] = input_data['DATE_OCC'].dt.day
        input_data['HOUR'] = input_data['DATE_OCC'].dt.hour
        input_data = input_data.drop(columns=['DATE_OCC'])

        # Codificar variables categóricas, igual que en el taller
        categorical_columns = ['Vict_Sex', 'Vict_Descent', 'Premis_Desc', 'Status']
        input_data = pd.get_dummies(input_data, columns=categorical_columns, drop_first=True)

        # Asegurar que todas las columnas necesarias están presentes
        for col in model_columns:
            if col not in input_data.columns:
                input_data[col] = 0

        # Ordenar las columnas para que coincidan con el modelo
        input_data = input_data[model_columns]

        # Hacer la predicción
        prediction = model.predict(input_data)
        primary_description = prediction[0]

        return CrimePredictionOutput(PRIMARY_DESCRIPTION=primary_description)

    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=500, detail="Internal Server Error")
